In [5]:
import pandas as pd
import numpy as np
import glob
import sys
import os

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [10]:
path_files = r"/content/drive/MyDrive/Data_M/Africain_Min_clusters/"

In [11]:
py_file_location = "/content/drive/MyDrive"
sys.path.append(os.path.abspath(py_file_location))

In [12]:
from function_tools import *

In [13]:
# récupérer les fichier du dossier contenant les fihcier de clusters
# glob aide à faire un trie par regex afin d'en choisir les fichiers nettoyés
list_file= glob.glob(f"{path_files}*")

In [14]:
list_file

['/content/drive/MyDrive/Data_M/Africain_Min_clusters/som_100',
 '/content/drive/MyDrive/Data_M/Africain_Min_clusters/som_9',
 '/content/drive/MyDrive/Data_M/Africain_Min_clusters/som_25',
 '/content/drive/MyDrive/Data_M/Africain_Min_clusters/som_4']

In [15]:
liste_clean , liste_brut = make_liste(list_file)

In [16]:
list_file

['/content/drive/MyDrive/Data_M/Africain_Min_clusters/som_4',
 '/content/drive/MyDrive/Data_M/Africain_Min_clusters/som_9',
 '/content/drive/MyDrive/Data_M/Africain_Min_clusters/som_25',
 '/content/drive/MyDrive/Data_M/Africain_Min_clusters/som_100']

In [17]:
clean_cluster= matrice(liste_clean)

In [18]:
brut_cluster= matrice(liste_brut)

In [19]:
len(brut_cluster[3])

100

In [21]:
data = make_df(brut_cluster, clean_cluster)

data_SOM : 
1.   index 0 : som4
2.   index 1: som9
3.   index 2: som25
4.   index 3: som100

In [ ]:
!pip install gensim==4.0
!pip install python-Levenshtein

In [23]:
from gensim.models import KeyedVectors

In [24]:
# récupérer les embeddings entrainés sur nos données
w2v_AF_Min = KeyedVectors.load('/content/drive/MyDrive/Data_M/w2v/w2vec_model_d300_1M_AF_Min')

In [25]:
import numpy as np

In [26]:
#encode un message
def encode(msg):
    return np.mean([w2v_AF_Min[word] for word in msg if word in w2v_AF_Min] or [np.zeros(300)], axis = 0)

In [27]:
w2v_AF_Min.similar_by_word("sale")

[('noir', 0.6970106959342957),
 ('sal', 0.6780790090560913),
 ('pute', 0.6673237085342407),
 ('singe', 0.6480235457420349),
 ('noir??', 0.6416674852371216),
 ('p*te', 0.6345035433769226),
 ('esclave,', 0.625322699546814),
 ('bougnoul,', 0.6246535778045654),
 ('raciste', 0.6195729970932007),
 ('noir,', 0.616439163684845)]

Encodage des différents set de clusters

In [29]:
len(clean_cluster)

4

In [31]:
dataset_encoded= liste_encoded(clean_cluster)

Entrainement des différents set de clusters

In [32]:
from sklearn.ensemble import IsolationForest

# definir le modèle ainsi que ses paramètres
if_model=IsolationForest(n_jobs=-1, random_state=42)


Créer une matrice qui stock les différents models d'IF correspondant aux différents clusters

In [34]:
def train_if (dataset):
  models=[[] for _ in range(len(dataset))]
  for i in range(len(dataset)):
    for n in range(len(dataset[i])):
      if_model=IsolationForest(n_jobs=-1, random_state=42)
      models[i].append(if_model.fit(dataset[i][n]))
  return models

Appliquer IF sur les clusters de Km

In [35]:
models = train_if(dataset_encoded)

Appliquer IF sur les clusters de SOM

Prédictions pour les différents modèles

In [37]:
def pred(dataset,model):
  preds=[[] for _ in range(len(dataset))]
  for i in range(len(dataset)):
    for n in range(len(dataset[i])):
      preds[i].append(model[i][n].predict(dataset[i][n]))
  return preds

In [38]:
pred= pred(dataset_encoded, models)

In [39]:
def add_pred_to_df(dataset,pred):
  for i in range(len(pred)):
    for n in range(len(pred[i])):
      dataset[i][n]['pred']= pred[i][n]

In [40]:
add_pred_to_df(data, pred)

In [42]:
# Trouver les outliers de chaque cluster
# ceux dont l'index est -1
def outliers(df):
  outliers=[]
  for i in range(len(df)):
    outliers.append(df[i]['tweet'].loc[df[i]['pred']==-1])
  return outliers

In [43]:
liste_cluster=["som_4","som_9","som_25","som_100"]

In [44]:
def save_outliers(outliers,string):
  for i in range(len(outliers)):
    with open(f"drive/MyDrive/af_min/"+string+f"/outliers_{i}", 'w') as f:
      for line in outliers[i]:
        f.write(line)

In [45]:
def loop_outliers(data, liste):
  for i in range(len(data)):
    save_outliers(outliers(data[i]),liste[i])

In [46]:
loop_outliers(data, liste_cluster)